# Solving Bellman Optimal Equations by solving non-linear equations

In [1]:
#%%
import numpy as np
import pdb

In [2]:
#%%initialisations
class Agent():
    def __init__(self):
        self.row = 5
        self.column = 5

        #states in the grid world
        self.state1 = np.empty((self.row,self.column))

        #value for each state
        self.state_value = {}
        value = 1
        for i in range(self.row):
            for j in range(self.column):
                self.state1[i,j] = value
                self.state_value[value] = None
                value +=1

        #action space
        self.action = {6: 'left', 7: 'right', 8: 'north', 9: 'south'}

    def action_state(self,action,state):
#        pdb.set_trace()
        reward = 0
        tot_rew = 0.0
        state1 = np.inf
        loop = True
        #leftmost corner condition
        if state in (self.state1[0,0],self.state1[4,0],\
                     self.state1[1,0],self.state1[2,0],\
                     self.state1[3,0]):
            if action == self.action[6]:
                reward = -1
                state1 = state
            if action == self.action[7]:
                reward = 0
                state1 = state + 1
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward  = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
            loop = False

        #right most condition
        if state == self.state1[0,4] or\
        state == self.state1[1,4] or \
        state == self.state1[2,4] or \
        state == self.state1[3,4] or \
        state == self.state1[4,4]:
            if action == self.action[6]:
                reward = 0
                state1 = state - 1
            if action == self.action[7]:
                reward = -1
                state1 = state
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
            loop = False


        #best conditions
        if state == self.state1[0,1]:
            action = self.action
            reward = 10
            state1 = state + 20
            loop = False
            tot_rew = reward

        if state  == self.state1[0,3]:
            action = self.action
            reward = 5
            state1 = state + 10
            tot_rew = reward
            loop = False

        elif loop == True:
            if action == self.action[6]:
                state1 = state - 1
                reward = 0
            if action == self.action[7]:
                state1 = state + 1
                reward = 0
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
        return state1, reward

In [4]:
#%%
import itertools

# Get all permutations of [1, 2, 3]
x = [0, 0.25, 1/3, 0.5, 2/3, 0.75, 1]

perm = [p for p in itertools.product(x, repeat=4)]

# Print the obtained permutations
temp = []
for i in list(perm):
    temp.append(i)

# Possible policy for each state.
mu_pi = []
for i in range(len(temp)):
    if np.sum(temp[i])==1:
        mu_pi.append(temp[i])

In [5]:
#%% Possible combination.
x_agent = Agent()

# Number of states.
num_states = x_agent.row*x_agent.column

# Number of actions and possible actions.
action_space = x_agent.action.keys()
num_actions = len(action_space)

# Discount factor
gamma = 0.9

V_s = np.zeros((num_states,len(mu_pi)))

# Action map
def act_map(mu):
    act_n = []

    act_space = np.arange(6,10)

    prob_act = []
    for i in range(len(mu)):
        if mu[i] != 0:
            act_n.append(act_space[i])
            prob_act.append(mu[i])

    return [prob_act,np.array(act_n)]

for pol in range(len(mu_pi)):
    if pol == 22:
        print('hold')
#        pdb.set_trace()
    #pdb.set_trace()
    [prob_act,ac] = act_map(mu_pi[pol])

    num_actions = len(ac)

    # States transited for each state.
    state_transit = np.zeros((num_states, num_actions))
    rew = np.zeros((num_states, num_actions))

    # Probability of transition.
    P_pi = np.zeros((num_states,num_states))

    update = 0
    for r in range(x_agent.row):
        for c in range(x_agent.column):
            # Prob of action.

            for l in range(len(ac)):
                [state_transit[update,l],rew[update,l]] =\
                x_agent.action_state(x_agent.action[ac[l]], x_agent.state1[r,c])

                # State
                s_temp = int(state_transit[update,l]) -1
                P_pi[update,s_temp] = P_pi[update,s_temp] + prob_act[l]
            update+=1

    #pdb.set_trace()
    #finding the reward
    reward = np.zeros((num_states,1))

    for i in range(num_states):
        for j in range(len(ac)):
            reward[i] = reward[i] + prob_act[j]*rew[i,j]

    # Finding value function
    I = np.identity(num_states)
    m = np.linalg.inv(np.eye(num_states) - gamma*P_pi)
    # Optimal value of the states.
    v_pi = np.matmul(m,reward)
    V_s[0:,pol] = np.reshape(v_pi,(num_states))

hold


In [6]:
#%% Optimal Policy.

pi_s = np.zeros((4,num_states))
v_final = np.zeros((num_states,1))
for i in range(num_states):
    v_final[i] = np.max(V_s[i,:])
    pi_s[:,i] = mu_pi[np.argmax(V_s[i,:])]

In [10]:
v_final.T

array([[ 7.35956481, 24.4194281 ,  8.68016838, 18.4501845 ,  5.25049028,
         6.45182012, 21.97748529,  7.59868837, 16.60516605,  4.63946605,
         5.38602382, 19.77973676,  6.1725332 , 14.94464945,  4.32719411,
         4.28716895, 17.80176308,  4.88740847, 13.4501845 ,  3.77732005,
         3.31337273, 16.02158677,  3.82844646, 12.10516605,  3.1324146 ]])